In [ ]:
# default_exp gan.learner

# gan.learner

> API details.

TODO maik: add simple synthetic data?!

In [ ]:
# hide
# export

import torch
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm, tnrange
from fastai.torch_core import set_seed
# from fastai.learner import Learner

In [ ]:
# first drafts for the actual learner and model classes

class W_Gan(nn.Module):
    def __init__(self, generator, discriminator, gen_optim, dis_optim)
        super(W_Gan, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.gen_optim = gen_optim
        self.dis_optim = dis_optim
        self.real_loss = []
        self.fake_loss = []
        
    def train_generator(self, z):
        # train the generator model
        self.generator.zero_grad()
        x_fake = self.generator(z)
        y_fake = self.discriminator(x_fake)
        loss = - y_fake.mean()
        loss.backward()
        self.gen_optim.step()
        return
    
    def train_discriminator(self, z, x_cont):
        # train the discriminator model
        self.discriminator.zero_grad()
        x_real = x_cont
        y_real = self.discriminator(x_real)
        real_loss = - y_real.mean()
        real_loss.backward()
        self.dis_optim.step()
        self.real_loss.append(real_loss.item())
        
        self.discriminator.zero_grad()
        x_fake = self.generator(z).detach()
        y_fake = self.discriminator(x_fake)
        fake_loss = y_fake.mean()
        fake_loss.backward()
        self.dis_optim.step()
        self.fake_loss.append(fake_loss.item())
        return

class GanLearner(Learner):
    def __init__(self, gan):
        super(GanLearner, self).__init__()
        # gan should contain a class which itself contains a generator and discriminator/critic class and combines them
        self.gan = gan
        
    def fit(self, dl, epochs, n_gen, n_dis)
        # train gan and store parameters and losses in given class
        for e in tqdm(range(epochs)):
            
            for z, x_cont, _ in dl:

                for _ in range(n_gen):
                    # issue - z will be the same for each iteration of the generator
                    # z = torch.randn_like(z) <- patch for this issue?
                    self.gan.train_generator(z)

                for _ in range(n_dis):
                    # same issue as above
                    # z = torch.randn_like(z)
                    self.gan.train_discriminator(z, x_cont)
        return

In [ ]:
# export
# TODO maik: should be a class with a constructur and a fit function
# TODO maik: remove col_name_ws
# TODO maik: can the device be removed?
def train_gan(epochs, dls, Dis, Gen, 
              dis_optimizer, gen_optimizer, 
              col_name_ws, device, 
              n_noise_samples, cols, 
              plot_every_n=5, plot_function=None,
              n_iter_dis=5,
              n_iter_gen=1,
             amount_clamp=0.01):
    set_seed(123123)
    n_samples = 1000
    rand_values = torch.rand(n_samples, n_noise_samples)

    dis_real_losses, dis_fake_losses, gen_losses = [],[],[]
    # Gen = Gen.to(device)
    # TODO maik: check if data is shuffled. e.g.:
#      if torch.allclose(dl.one_batch()[1], dl.one_batch()[1]):
#         raise ValueError(
#             "Dataset does not to be seemed shuffle, which is required for training."
#         )
    
    for epoch in tnrange(epochs, desc="epochs"):

        for i, (noise_sample, real_input_batch_cat,real_input_batch) in enumerate(dls[0]):
            noise_sample = noise_sample.to(device)
            real_input_batch = real_input_batch.to(device)
            minibatchsize = len(real_input_batch)

    #         ## train the discriminator (i-times per epoch):
            # TODO maik: should be configurable
            for i in range(n_iter_dis):
                Dis.zero_grad()
                ## train on real input
                # real_input_batch_cat is rather for the loss of the discriminator
                dis_real = Dis(None, real_input_batch)

                ## train on generated input
                dis_fake = Dis(None, Gen(real_input_batch_cat, noise_sample).detach())
                dis_loss = -(torch.mean(dis_real) - torch.mean(dis_fake))
                dis_loss.backward()
                dis_optimizer.step()


                ## clamp the discriminator parameter
                for p in Dis.parameters():
                    # TODO: what amount of clamp to use?
                    # TODO maik: should be configurable
                    p.data.clamp_(-amount_clamp, amount_clamp)

            dis_real_losses.append(dis_real.detach().mean())
            dis_fake_losses.append(dis_fake.detach().mean())
            # TODO maik: should be configurable
            for i in range(n_iter_gen):            
                Gen.zero_grad()
                gen_out = Gen(real_input_batch_cat, noise_sample)
                dis_gen = Dis(None, gen_out)
                gen_loss = -torch.mean(dis_gen)
                gen_loss.backward()
                gen_optimizer.step()

            gen_losses.append(gen_loss.detach().mean())

        disl = dis_loss.detach().mean()
        genl = gen_loss.detach().mean()
        # TODO maik: improve output of trainer
        print(f"{epoch} dis_loss {disl} gen_loss {genl} ")
        
        # TODO maik: should be configurable
        if epoch % plot_every_n == 0:
            if plot_function is not None:
                with torch.no_grad():
                    fake_samples = Gen(real_input_batch_cat, rand_values.to(device))
                plot_function(fake_samples)

    # TODO maik: rather store as attributes in the class?!
    return dis_real_losses, dis_fake_losses